In [ ]:
"""
PHASE 2: Build the Lineup Gate + Fix OddsAPI
"""

import pandas as pd
import requests
from datetime import datetime

# ============================================================
# 1. EXTRACT PP STATS FROM MONEYPUCK
# ============================================================

def get_pp_stats_from_moneypuck(csv_path):
    """Extract power play statistics from MoneyPuck data"""
    print("=" * 60)
    print("EXTRACTING PP STATS FROM MONEYPUCK")
    print("=" * 60)
    
    df = pd.read_csv(csv_path)
    
    # Check available situations
    print(f"\n📋 Available situations: {df['situation'].unique()}")
    
    # Filter for power play situations (5on4)
    pp_df = df[df['situation'] == '5on4'].copy()
    
    if len(pp_df) == 0:
        print("⚠️ No 5on4 data found. Checking for alternative PP labels...")
        # Try other possible PP labels
        for situation in df['situation'].unique():
            if 'pp' in situation.lower() or '5on4' in situation.lower():
                print(f"  Found: {situation}")
    else:
        print(f"\n✅ Found {len(pp_df)} players with PP data")
        
        # Create PP1 indicator (top PP time players by team)
        pp_df['pp_toi_pct'] = pp_df.groupby('team')['icetime'].rank(pct=True)
        pp_df['is_pp1'] = (pp_df['pp_toi_pct'] >= 0.6).astype(int)  # Top 40% = PP1
        
        print(f"\n📊 PP1 Players by Team (sample):")
        sample = pp_df[pp_df['is_pp1'] == 1][['name', 'team', 'icetime', 'I_F_xGoals']].head(10)
        print(sample.to_string())
        
        # Save PP stats
        pp_df.to_csv('moneypuck_pp_stats.csv', index=False)
        print(f"\n✅ Saved PP stats to: moneypuck_pp_stats.csv")
    
    # Get all situations data for base rates
    all_situations_df = df[df['situation'] == 'all'].copy()
    print(f"\n✅ Extracted 'all' situation data: {len(all_situations_df)} players")
    
    return all_situations_df, pp_df

# ============================================================
# 2. FIX ODDSAPI - TEST DIFFERENT MARKET NAMES
# ============================================================

def test_oddsapi_markets(api_key):
    """Test different market names to find the right one"""
    print("\n" + "=" * 60)
    print("TESTING ODDSAPI MARKET NAMES")
    print("=" * 60)
    
    base_url = "https://api.the-odds-api.com/v4/sports/icehockey_nhl/odds"
    
    # Test different market names
    market_names = [
        'player_goal_scorer',
        'player_anytime_goalscorer', 
        'player_anytime_goal_scorer',
        'h2h,spreads,totals',  # Start with basic markets
    ]
    
    for market in market_names:
        print(f"\n🧪 Testing market: {market}")
        params = {
            'apiKey': api_key,
            'regions': 'us',
            'markets': market,
            'oddsFormat': 'decimal'
        }
        
        try:
            response = requests.get(base_url, params=params)
            print(f"   Status: {response.status_code}")
            
            if response.status_code == 200:
                data = response.json()
                print(f"   ✅ SUCCESS - {len(data)} events returned")
                
                # Check if we have player props
                if data and 'bookmakers' in data[0]:
                    bookmakers = data[0]['bookmakers']
                    if bookmakers and 'markets' in bookmakers[0]:
                        markets = bookmakers[0]['markets']
                        print(f"   📊 Markets available: {len(markets)}")
                        if markets and 'outcomes' in markets[0]:
                            print(f"   📝 Sample outcomes: {markets[0]['outcomes'][:2]}")
                
                return market, data  # Return working market
            else:
                error_msg = response.json() if response.text else "No error message"
                print(f"   ❌ Failed: {error_msg}")
        
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    return None, None

# ============================================================
# 3. GET TODAY'S LINEUPS (Multiple Methods)
# ============================================================

def get_todays_lineups():
    """Get confirmed lineups for today's games"""
    print("\n" + "=" * 60)
    print("FETCHING TODAY'S LINEUPS")
    print("=" * 60)
    
    # Method 1: Get today's schedule
    schedule_url = "https://api-web.nhle.com/v1/schedule/now"
    response = requests.get(schedule_url)
    
    if response.status_code != 200:
        print("❌ Failed to get schedule")
        return None
    
    data = response.json()
    games_today = []
    
    if 'gameWeek' in data:
        for day in data['gameWeek']:
            if 'games' in day:
                games_today.extend(day['games'])
    
    print(f"\n🏒 Games today: {len(games_today)}")
    
    all_players_today = []
    
    for game in games_today:
        game_id = game.get('id')
        away_team = game.get('awayTeam', {}).get('abbrev')
        home_team = game.get('homeTeam', {}).get('abbrev')
        
        print(f"\n📋 Game: {away_team} @ {home_team} (ID: {game_id})")
        
        # Try to get roster from landing endpoint
        for team_type, team_abbrev in [('awayTeam', away_team), ('homeTeam', home_team)]:
            # Method 2: Use team roster endpoint
            team_roster_url = f"https://api-web.nhle.com/v1/roster/{team_abbrev}/current"
            roster_response = requests.get(team_roster_url)
            
            if roster_response.status_code == 200:
                roster_data = roster_response.json()
                
                # Extract forwards and defensemen
                for position_group in ['forwards', 'defensemen']:
                    if position_group in roster_data:
                        for player in roster_data[position_group]:
                            player_info = {
                                'game_id': game_id,
                                'team': team_abbrev,
                                'player_id': player.get('id'),
                                'name': f"{player.get('firstName', {}).get('default', '')} {player.get('lastName', {}).get('default', '')}",
                                'position': player.get('positionCode'),
                                'home_away': 'home' if team_type == 'homeTeam' else 'away'
                            }
                            all_players_today.append(player_info)
                
                print(f"  ✅ {team_abbrev}: {len(roster_data.get('forwards', [])) + len(roster_data.get('defensemen', []))} players")
            else:
                print(f"  ⚠️ {team_abbrev}: Could not fetch roster")
    
    if all_players_today:
        lineup_df = pd.DataFrame(all_players_today)
        print(f"\n✅ TOTAL PLAYERS IN LINEUPS TODAY: {len(lineup_df)}")
        print(f"\n📝 Sample lineup data:")
        print(lineup_df.head(10).to_string())
        
        # Save lineup
        lineup_df.to_csv('todays_lineups.csv', index=False)
        print(f"\n✅ Saved to: todays_lineups.csv")
        
        return lineup_df
    else:
        print("\n❌ Could not extract any lineups")
        return None

# ============================================================
# 4. CREATE FILTERED BETTING OPPORTUNITIES
# ============================================================

def create_lineup_gate(lineup_df, moneypuck_df, pp_df):
    """Filter betting opportunities to only players in today's lineup"""
    print("\n" + "=" * 60)
    print("CREATING LINEUP GATE")
    print("=" * 60)
    
    # Merge lineup with MoneyPuck stats
    # Need to normalize names for matching
    lineup_df['name_normalized'] = lineup_df['name'].str.lower().str.strip()
    moneypuck_df['name_normalized'] = moneypuck_df['name'].str.lower().str.strip()
    
    # Merge on player name
    merged = lineup_df.merge(
        moneypuck_df[['name_normalized', 'I_F_xGoals', 'I_F_shotsOnGoal', 'icetime', 'I_F_goals']],
        on='name_normalized',
        how='left'
    )
    
    print(f"\n✅ Matched {merged['I_F_xGoals'].notna().sum()} / {len(merged)} players to MoneyPuck stats")
    
    # Merge PP data
    if len(pp_df) > 0:
        pp_df['name_normalized'] = pp_df['name'].str.lower().str.strip()
        merged = merged.merge(
            pp_df[['name_normalized', 'is_pp1', 'icetime']],
            on='name_normalized',
            how='left',
            suffixes=('', '_pp')
        )
        print(f"✅ Matched {merged['is_pp1'].notna().sum()} players to PP data")
    
    # Save gated lineup
    merged.to_csv('todays_players_with_stats.csv', index=False)
    print(f"\n✅ Saved gated player list to: todays_players_with_stats.csv")
    
    return merged

# ============================================================
# RUN EVERYTHING
# ============================================================

if __name__ == "__main__":
    MONEYPUCK_CSV = "path/to/your/moneypuck.csv"  # UPDATE THIS
    ODDS_API_KEY = "your_api_key"  # UPDATE THIS
    
    print("\n🚀 STARTING PHASE 2: LINEUP GATE + DATA INTEGRATION\n")
    
    # Step 1: Extract PP stats from MoneyPuck
    all_stats_df, pp_df = get_pp_stats_from_moneypuck(MONEYPUCK_CSV)
    
    # Step 2: Fix OddsAPI
    working_market, odds_data = test_oddsapi_markets(ODDS_API_KEY)
    
    # Step 3: Get today's lineups
    lineup_df = get_todays_lineups()
    
    # Step 4: Create lineup gate
    if lineup_df is not None and all_stats_df is not None:
        gated_players = create_lineup_gate(lineup_df, all_stats_df, pp_df)
        
        print("\n" + "=" * 60)
        print("✅ LINEUP GATE COMPLETE")
        print("=" * 60)
        print(f"\nYou now have:")
        print(f"  1. todays_lineups.csv - {len(lineup_df)} players playing today")
        print(f"  2. todays_players_with_stats.csv - Players with MoneyPuck stats")
        print(f"  3. moneypuck_pp_stats.csv - PP statistics")
        print(f"\n🎯 Next: Only calculate probabilities for players in todays_players_with_stats.csv")